In [8]:
import mechwolf as mw
import time
import asyncio
import math
from datetime import timedelta
from loguru import logger
from sigfig import round

In [9]:
# Harvard pump code
import time
import asyncio
from loguru import logger
from mechwolf import _ureg

class HarvardSyringePump(mw.Pump):
    def __init__(self, syringe_volume, syringe_diameter, serial_port, name=None):
        super().__init__(name = name)
        self.serial_port = serial_port
        self.syringe_volume = _ureg.parse_expression(syringe_volume)
        self.syringe_diameter = _ureg.parse_expression(syringe_diameter)
        
    def __enter__(self):
        import aioserial

        self._ser = aioserial.AioSerial(
            self.serial_port,
            115200,
            parity = aioserial.PARITY_NONE,
            stopbits = 1,
            timeout = 1,
            write_timeout = 1,)
        syringe_volume_ml = self.syringe_volume.to(_ureg.ml).magnitude 
        syringe_diameter_mm = self.syringe_diameter.to(_ureg.mm).magnitude
        self._ser.write(f'svolume {syringe_volume_ml} ml\r'.encode())
        self._ser.write(f'diameter {syringe_diameter_mm}\r'.encode())

        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.rate = _ureg.parse_expression("0 mL/min")
        self._ser.write(b'stop\r') 
        
      
        del self._ser

    async def _update(self):
        rate_mlmin = self.rate.to(_ureg.ml / _ureg.min).magnitude
        if rate_mlmin == 0:
            self._ser.write(b'stop\r') 
        else:
            self._ser.write(f'irate {rate_mlmin} m/m\r'.encode())
            self._ser.write(b'irun\r')

In [10]:
pump_1 = HarvardSyringePump("24mL", "20.05mm", serial_port = '/dev/tty.usbmodemD4038611', name = "Pump 1")
pump_2 = HarvardSyringePump("24mL", "20.05mm", serial_port = '/dev/tty.usbmodemD4038591', name = "Pump 2")
pump_3 = HarvardSyringePump("24mL", "20.05mm", serial_port = '/dev/tty.usbmodemD4038601', name = "Pump 3")

### Syringe, Tubing, and Mixer Information

In [ ]:
#reagent vessels
reagent_vessel_1 = mw.Vessel("Diacetyl Rhamnal", name = "Starting Material")
reagent_vessel_2 = mw.Vessel("NaOMe", name = "NaOMe + MeOH in DMF")
reagent_vessel_3 = mw.Vessel("TBSCl", name = "TBSCl + DMAP")
reagent_vessel_4 = mw.Vessel("KHMDS", name = "KHMDS in DMF")
reagent_vessel_5 = mw.Vessel("BnBr", name = "BnBr in DMF")
product_vessel = mw.Vessel("C3 benzylated rhamnal", name = "product")

#tubing
def big_tube(length):
    return mw.Tube(length = length, ID = "0.062 in", OD = "1/8 in", material = "PFA")

sugar = big_tube(length = "1 foot")
NaOMe = big_tube(length = "1 foot")
deacetylation = big_tube(length = "16 foot")
TBSCl = big_tube(length = "1 foot")
protection = big_tube(length = "32 foot")
NaHMDS = big_tube(length = "17 foot")
deprotonation = big_tube(length = "18 foot")
BnBr = big_tube(length = "1 foot")
benzylation = big_tube(length = "64 foot")

#mixers
def Tmixer(name):
    return mw.TMixer(name = name)

T_1 = Tmixer(deacetylation)
T_2 = Tmixer(protection)
T_3 = Tmixer(deprotonation)
T_4 = Tmixer(benzylation) 

### Apparatus 

In [ ]:
# apparatus 
A = mw.Apparatus("Telescoped Sequence")

A.add(pump_1, reagent_vessel_1, sugar)
A.add(pump_1, reagent_vessel_2, NaOMe)
A.add(reagent_vessel_1, T_1, sugar)
A.add(reagent_vessel_2, T_1, NaOMe)
A.add(T_1, T_2, deacetylation)


A.add(pump_2, reagent_vessel_3, TBSCl)
A.add(pump_2, reagent_vessel_4, NaHMDS)
A.add(reagent_vessel_3, T_2, TBSCl)
A.add(reagent_vessel_4, T_3, NaHMDS)
A.add(T_2, T_3, protection)
A.add(T_3, T_4, deprotonation)


A.add(pump_3, reagent_vessel_5, BnBr)
A.add(reagent_vessel_5, T_4, BnBr)
A.add(T_4, product_vessel, benzylation)

A.describe()
A.visualize()

### Define Protocol

In [ ]:
P = mw.Protocol(A)

switch = timedelta(minutes = 1)

current = timedelta(seconds = 0)

pump_1_initial = timedelta(minutes = 3.8)
P.add(pump_1, start = current,
              duration = pump_1_initial, rate = "0.5mL/min")

current += pump_1_initial + switch

deacetylation = timedelta(minutes = 7.7) 
P.add(pump_1, start = current, 
              duration = deacetylation + NaHMDS_1, rate = "0.5mL/min")

current += deacetylation

NaHMDS_1 = timedelta(minutes = 4)
P.add(pump_2, start = current,
              duration = NaHMDS_1, rate = "1mL/min")

current += NaHMDS_1 + switch

NaHMDS_2 = timedelta(minutes = 4) 
P.add(pump_1, start = current, 
              duration = NaHMDS_2, rate = "0.5mL/min")
P.add(pump_2, start = current, 
              duration = NaHMDS_2, rate = "1mL/min")
      
current += NaHMDS_2 + switch 

protection_deprotonation = timedelta(minutes = 6.4)    
P.add(pump_1, start = current, 
              duration = protection_deprotonation + BnBr_1, rate = "0.5mL/min")
P.add(pump_2, start = current, 
              duration = protection_deprotonation + BnBr_1, rate = "1mL/min")

current += protection_deprotonation

BnBr_1 = timedelta(minutes = 5.67) 
P.add(pump_3, start = current + protection_deprotonation, 
              duration = BnBr_1, rate = "3mL/min")

current += BnBr_1 + switch

benzylation = timedelta(minutes = 6.7)
P.add(pump_1, start = current, 
              duration = benzylation, rate = "0.5mL/min")
P.add(pump_2, start = current, 
              duration = benzylation, rate = "1mL/min")
P.add(pump_3, start = current, 
              duration = benzylation, rate = "3mL/min")

current += benzylation

print(f'TOTAL TIME: {current}')
P.execute(confirm = True)